In [1]:
!pip install -q gwpy --quiet

In [2]:
%%capture
%run /content/load_data.ipynb 

## TFIDF Classifier

In [3]:
df_train

,index,comment,rate,clean_comment,no_stopwords_comment,no_enligh_comment,stemmed_version,lemmatized_version,label
0,2587,پردازنده های Core i5 و Core i3 نیز ذاتا دو هست...,0.0,"[پردازنده, های, Core, i5, و, Core, i3, نیز, ذا...","[پردازنده, های, Core, i5, Core, i3, ذاتا, هسته...","[پردازنده, های, 5, و, 3, نیز, ذاتا, دو, هسته, ...","[پردازنده, , Core, i5, و, Core, i3, نیز, ذاتا,...","[پردازنده, های, Core, i5, و, Core, i3, نیز, ذا...",neutral
1,22591,سلام به دوستای عزیزم \r\nعزاداری هاتون قبول باشه,1.0,"[سلام, به, دوستای, عزیزم, عزاداری, هاتون, قبول...","[سلام, دوستای, عزیزم, عزاداری, هاتون, قبول, باشه]","[سلام, به, دوستای, عزیزم, عزاداری, هاتون, قبول...","[سلا, به, دوستا, عزیز, عزادار, هاتون, قبول, باشه]","[سلام, به, دوستای, عزیز, عزادار, هاتون, قبول, ...",positive
2,141037,کلا پولتون رو دور نریزیزد,-1.0,"[کلا, پولتون, رو, دور, نریزیزد]","[کلا, پولتون, نریزیزد]","[کلا, پولتون, رو, دور, نریزیزد]","[کلا, پولتون, رو, دور, نریزیزد]","[کلا, پولتون, رو, دور, نریزیزد]",negative
3,58593,از صمیم قلب امیدوارم دایانا با کارن بمونه و پو...,1.0,"[از, صمیم, قلب, امیدوارم, دایانا, با, کارن, بم...","[صمیم, قلب, امیدوارم, دایانا, کارن, بمونه, پور...","[از, صمیم, قلب, امیدوارم, دایانا, با, کارن, بم...","[از, صم, قلب, امیدوار, دایانا, با, کارن, بمونه...","[از, صمیم, قلب, امیدوار, دایانا, با, کارن, بمو...",positive
4,5712,آنطور که اپل ادعا می کند آیپاد شافل دارای طراح...,1.0,"[آنطور, که, اپل, ادعا, می, کند, آیپاد, شافل, د...","[آنطور, اپل, ادعا, آیپاد, شافل, طراحی, العاده,...","[آنطور, که, اپل, ادعا, می, کند, آیپاد, شافل, د...","[آنطور, که, اپل, ادعا, م, کند, آیپاد, شافل, دا...","[آنطور, که, اپل, ادعا, می, کند, آیپاد, شافل, د...",positive
...,...,...,...,...,...,...,...,...,...
795,6115,به هر صورت همانند دیگر دوربین‌های سونی سایبرشا...,1.0,"[به, هر, صورت, همانند, دیگر, دوربینهای, سونی, ...","[صورت, همانند, دوربینهای, سونی, سایبرشات, HX, ...","[به, هر, صورت, همانند, دیگر, دوربینهای, سونی, ...","[به, هر, صور, همانند, دیگر, دوربین, سون, سایبر...","[به, هر, صورت, همانند, دیگر, دوربین, سون, سایب...",positive
796,5766,این دقیقا همان مقداریست که از دوربین یک گوشی ب...,0.0,"[این, دقیقا, همان, مقداریست, که, از, دوربین, ی...","[دقیقا, مقداریست, دوربین, گوشی, پردازندهی, تک,...","[این, دقیقا, همان, مقداریست, که, از, دوربین, ی...","[این, دقیقا, ه, مقداریس, که, از, دوربین, یک, گ...","[این, دقیقا, همان, مقداریست, که, از, دوربین, ی...",neutral
797,62772,خیار تازه نبود و سیب زرد لک داشت,-1.0,"[خیار, تازه, نبود, و, سیب, زرد, لک, داشت]","[خیار, تازه, سیب, زرد, لک]","[خیار, تازه, نبود, و, سیب, زرد, لک, داشت]","[خیار, تازه, نبود, و, سیب, زرد, لک, دا]","[خیار, تازه, نبود, و, سیب, زرد, لک, داشت]",negative
798,18359,کیفیت مزه و ارسال بینظیر,-1.0,"[کیفیت, مزه, و, ارسال, بینظیر]","[کیفیت, مزه, ارسال, بینظیر]","[کیفیت, مزه, و, ارسال, بینظیر]","[کیف, مزه, و, ارسال, بینظیر]","[کیفیت, مزه, و, ارسال, بینظیر]",negative


In [14]:
df_train

,index,comment,rate,clean_comment,no_stopwords_comment,no_enligh_comment,stemmed_version,lemmatized_version,label
0,2587,پردازنده های Core i5 و Core i3 نیز ذاتا دو هست...,0.0,"[پردازنده, های, Core, i5, و, Core, i3, نیز, ذا...","[پردازنده, های, Core, i5, Core, i3, ذاتا, هسته...","[پردازنده, های, 5, و, 3, نیز, ذاتا, دو, هسته, ...","[پردازنده, , Core, i5, و, Core, i3, نیز, ذاتا,...","[پردازنده, های, Core, i5, و, Core, i3, نیز, ذا...",neutral
1,22591,سلام به دوستای عزیزم \r\nعزاداری هاتون قبول باشه,1.0,"[سلام, به, دوستای, عزیزم, عزاداری, هاتون, قبول...","[سلام, دوستای, عزیزم, عزاداری, هاتون, قبول, باشه]","[سلام, به, دوستای, عزیزم, عزاداری, هاتون, قبول...","[سلا, به, دوستا, عزیز, عزادار, هاتون, قبول, باشه]","[سلام, به, دوستای, عزیز, عزادار, هاتون, قبول, ...",positive
2,141037,کلا پولتون رو دور نریزیزد,-1.0,"[کلا, پولتون, رو, دور, نریزیزد]","[کلا, پولتون, نریزیزد]","[کلا, پولتون, رو, دور, نریزیزد]","[کلا, پولتون, رو, دور, نریزیزد]","[کلا, پولتون, رو, دور, نریزیزد]",negative
3,58593,از صمیم قلب امیدوارم دایانا با کارن بمونه و پو...,1.0,"[از, صمیم, قلب, امیدوارم, دایانا, با, کارن, بم...","[صمیم, قلب, امیدوارم, دایانا, کارن, بمونه, پور...","[از, صمیم, قلب, امیدوارم, دایانا, با, کارن, بم...","[از, صم, قلب, امیدوار, دایانا, با, کارن, بمونه...","[از, صمیم, قلب, امیدوار, دایانا, با, کارن, بمو...",positive
4,5712,آنطور که اپل ادعا می کند آیپاد شافل دارای طراح...,1.0,"[آنطور, که, اپل, ادعا, می, کند, آیپاد, شافل, د...","[آنطور, اپل, ادعا, آیپاد, شافل, طراحی, العاده,...","[آنطور, که, اپل, ادعا, می, کند, آیپاد, شافل, د...","[آنطور, که, اپل, ادعا, م, کند, آیپاد, شافل, دا...","[آنطور, که, اپل, ادعا, می, کند, آیپاد, شافل, د...",positive
...,...,...,...,...,...,...,...,...,...
795,6115,به هر صورت همانند دیگر دوربین‌های سونی سایبرشا...,1.0,"[به, هر, صورت, همانند, دیگر, دوربینهای, سونی, ...","[صورت, همانند, دوربینهای, سونی, سایبرشات, HX, ...","[به, هر, صورت, همانند, دیگر, دوربینهای, سونی, ...","[به, هر, صور, همانند, دیگر, دوربین, سون, سایبر...","[به, هر, صورت, همانند, دیگر, دوربین, سون, سایب...",positive
796,5766,این دقیقا همان مقداریست که از دوربین یک گوشی ب...,0.0,"[این, دقیقا, همان, مقداریست, که, از, دوربین, ی...","[دقیقا, مقداریست, دوربین, گوشی, پردازندهی, تک,...","[این, دقیقا, همان, مقداریست, که, از, دوربین, ی...","[این, دقیقا, ه, مقداریس, که, از, دوربین, یک, گ...","[این, دقیقا, همان, مقداریست, که, از, دوربین, ی...",neutral
797,62772,خیار تازه نبود و سیب زرد لک داشت,-1.0,"[خیار, تازه, نبود, و, سیب, زرد, لک, داشت]","[خیار, تازه, سیب, زرد, لک]","[خیار, تازه, نبود, و, سیب, زرد, لک, داشت]","[خیار, تازه, نبود, و, سیب, زرد, لک, دا]","[خیار, تازه, نبود, و, سیب, زرد, لک, داشت]",negative
798,18359,کیفیت مزه و ارسال بینظیر,-1.0,"[کیفیت, مزه, و, ارسال, بینظیر]","[کیفیت, مزه, ارسال, بینظیر]","[کیفیت, مزه, و, ارسال, بینظیر]","[کیف, مزه, و, ارسال, بینظیر]","[کیفیت, مزه, و, ارسال, بینظیر]",negative


### Base Line

In [4]:
base_line = df_train[df_train['label']=='positive'].shape[0]/df_train.shape[0]
base_line

0.41875

In [5]:
vectorizer = TfidfVectorizer(analyzer='word', min_df=0.05, max_df =0.90, ngram_range = (1,1), max_features=10000)

no_stopwords_comment_train = df_train['no_stopwords_comment'].apply(lambda comment:" ".join(comment))
train_data_features = vectorizer.fit_transform(no_stopwords_comment_train)

print(train_data_features.shape)

(800, 8)


#### Data snooping ALERT: we should transforom not fit again

In [7]:
no_stopwords_comment_eval = df_eval['no_stopwords_comment'].apply(lambda comment:" ".join(comment))
eval_data_features = vectorizer.transform(no_stopwords_comment_eval)

#### Feature Names

In [8]:
vectorizer.get_feature_names()[:30]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



['استفاده', 'اين', 'داره', 'های', 'واقعا', 'کتاب', 'کیفیت', 'گوشی']

In [9]:
# Load model

solvers= ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
multi_classes = ['multinomial', 'ovr']

model = LogisticRegression(penalty='l2', random_state=0, solver='lbfgs', max_iter=1000, multi_class='ovr')
# Train model
model.fit(train_data_features, df_train['label'])


LogisticRegression(max_iter=1000, multi_class='ovr', random_state=0)

### evaluation 

In [10]:
y_eval_perd = model.predict(eval_data_features)

#### Accuracy Score

In [11]:
print(accuracy_score(df_eval['label'], y_eval_perd))

0.385


#### Confusion Matrix

In [12]:
print(confusion_matrix(df_eval['label'], y_eval_perd))

[[ 0  5 50]
 [ 0 16 61]
 [ 0  7 61]]


#### Classification Report

In [13]:
print(classification_report(df_eval['label'], y_eval_perd))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        55
     neutral       0.57      0.21      0.30        77
    positive       0.35      0.90      0.51        68

    accuracy                           0.38       200
   macro avg       0.31      0.37      0.27       200
weighted avg       0.34      0.39      0.29       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

